In [1]:
#Libraries
import numpy as np
import laspy as lp
import pylas
import scipy
from scipy.spatial import cKDTree
import math
import matplotlib.pyplot as plt
from scipy.spatial import distance
import sys
from collections import Counter
from sklearn.cluster import DBSCAN
import pandas as pd
import geopandas as gpd
from scipy.spatial import KDTree

# 1. LOAD the data and Lower the point density

In [5]:
# order 22_medium, 16_down ,10_medium-2, 22_small , 16_up, 10_small_3

In [ ]:
# order test 3-4 small_lot, 10_medium, 03_small_second_ahn , 03_small, 15_ahn, en2 small 1, en2 small 2

In [ ]:
radius = 0.8 # the radius on which we count the point density in x and y for each point (the parameter used for density in 2D calculation)

f = lp.file.File('Thesis_Random_Forest/AHN_test/Final_train_4_to_3/37EN2_10_small_2_AHN4.las',mode="r")
f2 = lp.file.File('Thesis_Random_Forest/AHN_test/Final_train_4_to_3/37EN2_10_small_2_AHN3.las', mode="r")

# concatenate the file coordinates
coord = np.c_[f.x, f.y, f.z, f.red, f.green, f.blue, f.intensity,f.num_returns, f.classification]
coord2 = np.c_[f2.x, f2.y, f2.z, f2.red, f2.green, f2.blue, f2.intensity,f2.num_returns, f2.classification]

# now we have smaller coordiante values to deal with

xi = (f.x).astype('int')
yi = (f.y).astype('int')
zi = (f.z).astype('int')
xi2 = (f2.x).astype('int')
yi2 = (f2.y).astype('int')
zi2 = (f2.z).astype('int')

# Reduce the data! get a flat voxel index ii
# lowers the number of points feed to the algorithm
ii = xi + yi * xi.max() + zi * yi.max() * xi.max()
_, sl = np.unique(ii, return_index=True)
coord = coord[sl, :]

#
ii2 = xi2 + yi2 * xi2.max() + zi2 * yi2.max() * xi2.max()
_, sl2 = np.unique(ii2, return_index=True)
coord2 = coord2[sl2, :]
print("Initially we had", len(f.x), len(f2.x), "points. But, after lowering the point denity, the number of remainig points is", len(coord), len(coord2)) 

# 2. Match the data from 2 different epoch

In [121]:
# Construct KDTree from array2
tree = KDTree(coord2[:, :2])

# Find the closest point in array2 to each point in array1
dist, ind = tree.query(coord[:, :2])

# Reorder array2 based on ind
array2_reordered = coord2[ind]


In [123]:
df1 = pd.DataFrame(coord, columns=['xAHN3', 'yAHN3', 'zAHN3', 'redAHN3', 'greenAHN3', 'blueAHN3', 'intensityAHN3','rtnumAHN3', 'labelAHN3'])
df2 = pd.DataFrame(array2_reordered, columns=['x', 'y', 'z', 'red', 'green', 'blue', 'intensity','rtnum', 'label'])

In [124]:
df3 = pd.DataFrame(
    np.column_stack([df1,df2]),
    columns=df1.columns.append(df2.columns)
)

In [125]:
new_gdf_2 = df3
new_gdf_2

,xAHN3,yAHN3,zAHN3,redAHN3,greenAHN3,blueAHN3,intensityAHN3,rtnumAHN3,labelAHN3,x,y,z,red,green,blue,intensity,rtnum,label
0,89643.746,448178.177,-2.034,40960.0,37888.0,34304.0,850.0,1.0,2.0,89643.742,448178.789,-1.869,40192.0,40448.0,39168.0,152.0,1.0,2.0
1,89634.844,448200.215,-2.058,40448.0,36352.0,33024.0,960.0,1.0,2.0,89634.828,448199.648,-1.867,39168.0,39168.0,37120.0,149.0,1.0,2.0
2,89630.824,448210.342,-2.050,37632.0,33280.0,31232.0,918.0,1.0,2.0,89630.992,448210.242,-1.904,40704.0,38144.0,37888.0,159.0,1.0,2.0
3,89624.938,448149.270,-1.689,19456.0,18944.0,19712.0,1030.0,1.0,2.0,89624.773,448149.996,-1.634,48128.0,48384.0,46848.0,245.0,1.0,2.0
4,89625.770,448149.886,-1.565,26624.0,26112.0,26368.0,910.0,1.0,2.0,89625.676,448149.643,1.337,33792.0,34560.0,32768.0,219.0,1.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8562,89669.008,448215.980,8.214,50688.0,38144.0,30464.0,1272.0,1.0,6.0,89668.719,448215.708,-1.815,52480.0,42496.0,35072.0,227.0,1.0,2.0
8563,89667.730,448216.000,8.018,53504.0,38144.0,30976.0,1308.0,1.0,6.0,89667.785,448215.728,-1.818,51968.0,41728.0,35328.0,236.0,1.0,2.0
8564,89668.668,448216.135,8.169,52992.0,39424.0,31232.0,1318.0,1.0,6.0,89668.719,448215.708,-1.815,52480.0,42496.0,35072.0,227.0,1.0,2.0
8565,89669.945,448216.208,8.012,49920.0,40448.0,33792.0,1017.0,2.0,6.0,89669.953,448216.706,-1.783,51456.0,41984.0,37120.0,210.0,1.0,2.0


# 3. Generate the change_label target values

In [126]:
new_gdf_2["change_label"] = new_gdf_2.apply(lambda x: 1 if x["labelAHN3"] == 6 and x["label"] !=6 else 2 if x["label"] == 6 and x["labelAHN3"] != 6 else 0, axis = 1)

In [128]:
# get the data in an array format
data_array = new_gdf_2.values

# Calculate point density on 2D

In [130]:
# Input data
points_ahn3 = data_array[:, 0:3]

# Create KDTree from points
kd_tree11 = KDTree(points_ahn3)

# Array to store the number of points in the buffer for each point
count1 = np.zeros(len(points_ahn3), dtype=int)

# Loop over each point and find points in the buffer
for i, p in enumerate(points_ahn3):
    idx = kd_tree11.query_ball_point(p, radius)
    count1[i] = len(idx) - 1

In [131]:
# Input data
points_ahn4 = data_array[:, 9:11]

# Create KDTree from points
kd_tree22 = KDTree(points_ahn4)

# Array to store the number of points in the buffer for each point
count2 = np.zeros(len(points_ahn4), dtype=int)

# Loop over each point and find points in the buffer
for i, p in enumerate(points_ahn4):
    idx = kd_tree22.query_ball_point(p, radius)
    count2[i] = len(idx) - 1

# 4. Prepare and save the arrays for deep learning models

In [132]:
# extract features and targets
ahn3_features = data_array[:, 2:8]  # z until number of returns
ahn4_features = data_array[:, 11:17]  # z of second data until number of returns of second data

In [ ]:
#  Subtract and add features from 2 epochs with each other to get a single input
result_substraction = ahn3_features - ahn4_features
result_addition = ahn3_features + ahn4_features
# point density in 2d
sum_count = count1 + count2
sub_count = count1 - count2

In [136]:
# For each tile we insert an unique value
value1 = 0
column0 = np.full_like(data_array[:,1], value1)

In [21]:
# Save the coordinates alongside the new merged features
ahn_train_data = np.c_[data_array[:, 0:3], result_addition, result_substraction, sum_count, sub_count, column0]

In [22]:
# the change label
targets = data_array[:, 18]

# 5. The cluster cleaning code

In [ ]:
point_labels = targets
# Go through each unique label
for label in np.unique(targets):
    
    # Get all points with the current label
    current_points = ahn_train_data[:,0:3][targets == label]
    
    # Perform DBSCAN clustering on the current set of points
    clustering = DBSCAN(eps=2.5, min_samples=1).fit(current_points)
    
    # Get a list of unique cluster labels for the current set of points
    current_unique_labels = np.unique(clustering.labels_)
    
    # Create a boolean array to store which points belong to small clusters
    small_cluster_points = np.zeros(current_points.shape[0], dtype=bool)
    
    # Go through each unique label for the current set of points
    for current_label in current_unique_labels:
        
        # Skip the label for noise (-1)
        if current_label == -1:
            continue
        
        # Get all points in the current cluster
        current_cluster = clustering.labels_ == current_label
        
        # Check if the number of points in the cluster is less than 3
        if current_cluster.sum() < 80:
            
            # If the cluster is too small, mark all points in the cluster as belonging to small clusters
            small_cluster_points[current_cluster] = True
    
    # Change the labels of all points in small clusters to 0
    current_labels = point_labels[point_labels == label].copy()
    current_labels[small_cluster_points] = 0
    point_labels[point_labels == label] = current_labels

# Print the updated labels
print(point_labels)

# Save the results 

In [ ]:
# Save the training/testing tile as a las file to visually check the results

In [161]:
print_coord = ahn_train_data
fn = "Train_density_.las"

vals = np.linspace(0, 1, 100)
np.random.shuffle(vals)
cmap = plt.cm.colors.ListedColormap(plt.cm.tab20(vals))
header = lp.header.Header()
header.data_format_id = 2
fp = lp.file.File(fn, mode = 'w', header = header)
fp.header.scale = [0.01, 0.01, 0.01]
fp.header.offset = [min(print_coord[:,0]), min(print_coord[:,1]), min(print_coord[:,2])]
fp.x = print_coord[:, 0]
fp.y = print_coord[:, 1]
fp.z = print_coord[:, 2]
fp.pt_src_id = point_labels
#fp.intensity = intensity
fp.close()

In [162]:
# if you want to stack multiple training data into one to train algorithm with more than just one tile
AHN_train_stack = np.vstack((ahn_train_data, ahn_train_data2, ahn_train_data3, ahn_train_data4, ahn_train_data5, ahn_train_data6, ahn_train_data7))
AHN_train_target_stack = np.concatenate((point_labels, point_labels2, point_labels3, point_labels4, point_labels5, point_labels6, point_labels7), axis=0)
# else just save the singular file on the cells below

In [ ]:
# x,y,z, - addition(z,r,g,b,in,rtn), sub(z,r,g,b,in,rtn) ,sum 2d, sub 2d, label

In [141]:
np.save('AHN_test_FEATURES_FINAL_FINAL_density.npy', AHN_train_stack)
np.save('AHN_test_TARGET_FINAL_FINAL_density.npy', AHN_train_target_stack)

In [163]:
np.save('AHN_train_FEATURES_FINAL_FINAL_density.npy', AHN_train_stack)
np.save('AHN_train_TARGET_FINAL_FINAL_density.npy', AHN_train_target_stack)